In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../")
import geopandas as gpd
from src.organized_datasets_creation.utils import resolve_nominatim_city_name
from src.graph_layering.graph_layer_creator import GraphLayerController
import pandas as pd
import os
from src.graph_layering.graph_layer_creator import SourceType
import warnings
from src.graph_layering.create_hetero_data import create_hetero_data
from sklearn.preprocessing import OneHotEncoder
from typing import cast
from tqdm import tqdm

import wandb.util
import wandb
import os

import numpy as np
from src.graph.create_osmnx_graph import OSMnxGraph
import json
from shapely.geometry import Point
from joblib import dump


from datetime import datetime
from sklearn.metrics import f1_score, roc_auc_score
from wandb.util import generate_id
from sklearn.linear_model import LogisticRegression
from src.training.train import train
from sklearn.preprocessing import StandardScaler

In [3]:
WANDB_API_KEY = os.environ.get("WANDB_API_KEY", None)
assert (
    WANDB_API_KEY is not None
), "WANDB_API_KEY is not set, did you forget it in the config file?"

In [4]:
# general settings
ORGANIZED_HEXES_LOCATION = "/home/grymar/studia/gradient/data/organized-hexes"
ORGANIZED_GRAPHS_LOCATION = "/home/grymar/studia/gradient/data/organized_graphs"
OSMNX_ALL_ATTRIBUTES_LOCATION = (
    "/home/grymar/studia/gradient/data/osmnx_attributes.json"
)
HEX_FI_LOCATION = "/home/grymar/studia/gradient/data/downstream_tasks/feature_importance"

# downstream task settings
AIRBNB_LOCATION = "/home/grymar/studia/gradient/data/downstream_tasks/airbnb"
TRAIN_SAVE_DIR = "/home/grymar/tmp/"

SWEEP_RUNS_COUNT = 1

ATTRIBUTES_CONFIGURATIONS = [
    # {
    #     "USE_ORTOPHOTO": False,
    #     "USE_HEXES_ATTRS": True,
    #     "USE_OSMNX_ATTRS": True,
    # },
    {
        "USE_ORTOPHOTO": False,
        "USE_HEXES_ATTRS": True,
        "USE_OSMNX_ATTRS": False,
    },
    # {
    #     "USE_ORTOPHOTO": False,
    #     "USE_HEXES_ATTRS": False,
    #     "USE_OSMNX_ATTRS": True,
    # },
]

HEX_ATTRS_FI = [

    {
        "NUM_FEATURES": 20,
        "IN_PERCENT": False
    },
    
    {
        "NUM_FEATURES": 20,
        "IN_PERCENT": True
    },

    {
        "NUM_FEATURES": 50,
        "IN_PERCENT": False
    },

    {
        "NUM_FEATURES": 50,
        "IN_PERCENT": True
    },

]


WANDB_SWEEP_PARAMS_GRAPH_DATA = {
    "method": "bayes",
    "metric": {"name": "mean_f1", "goal": "maximize"},
    "parameters": {
        "hidden_channels": {"values": [10, 20, 30, 40, 50]},
        "learning_rate": {
            "distribution": "log_uniform_values",
            "min": 1e-5,
            "max": 1e-2,
        },
        "epochs": {
            "distribution": "int_uniform",
            "min": 10,
            "max": 20,
        },
        "num_conv_layers": {"values": [1, 2, 3, 4, 5]},
        "lin_layer_size": {"values": [8, 16, 32, 64, 128]},
        "num_lin_layers": {"values": [0, 1, 2, 3, 4]},
        "weight_decay": {
            "distribution": "log_uniform_values",
            "min": 1e-5,
            "max": 1e-2,
        },
    },
}

WANDB_SWEEP_PARAMS_TABULAR_DATA = {
    "method": "bayes",
    "metric": {"name": "mean_f1", "goal": "maximize"},
    "parameters": {
        "solver_penalty": {
            "values": [
                "lbfgs;l2",
                "liblinear;l1",
                "liblinear;l2",
                "newton-cg;l2",
                "newton-cholesky;l2",
                "sag;l2",
                "saga;elasticnet",
                "saga;l1",
                "saga;l2",
            ]
        },
        "C": {
            "distribution": "log_uniform_values",
            "min": 1e-5,
            "max": 1,
        },
    },
}


# Load airbnb data


In [5]:
def create_gdf_airbnb_price(data):
    gdf = gpd.GeoDataFrame(
        data.drop(["latitude", "longitude"], axis=1),
        geometry=gpd.points_from_xy(x=data["longitude"], y=data["latitude"]),
        crs="EPSG:4326",
    )
    gdf_price = gdf[["geometry", "price"]]
    gdf_price.reset_index(inplace=True, drop=True)

    return gdf_price

In [6]:
airbnb_ny = pd.read_csv(f"{AIRBNB_LOCATION}/airbnb_new_york_cleaned.csv")
airbnb_ny = create_gdf_airbnb_price(airbnb_ny)
airbnb_ny["mie_nazwa"] = "new_york"
airbnb_ny

,geometry,price,mie_nazwa
0,POINT (-73.95512 40.68535),72.0,new_york
1,POINT (-73.99454 40.66265),253.0,new_york
2,POINT (-73.95342 40.70935),81.0,new_york
3,POINT (-73.94255 40.80107),65.0,new_york
4,POINT (-73.94759 40.78778),60.0,new_york
...,...,...,...
23756,POINT (-74.00952 40.71445),721.0,new_york
23757,POINT (-74.00940 40.71647),721.0,new_york
23758,POINT (-74.01149 40.71617),1080.0,new_york
23759,POINT (-73.94935 40.78102),198.0,new_york


In [7]:
airbnb_st = pd.read_csv(f"{AIRBNB_LOCATION}/airbnb_seattle_cleaned.csv")
airbnb_st = create_gdf_airbnb_price(airbnb_st)
airbnb_st["mie_nazwa"] = "seattle"
airbnb_st

,geometry,price,mie_nazwa
0,POINT (-122.33629 47.65444),99.0,seattle
1,POINT (-122.31937 47.55017),72.0,seattle
2,POINT (-122.38663 47.55495),166.0,seattle
3,POINT (-122.38607 47.55627),125.0,seattle
4,POINT (-122.37196 47.67947),81.0,seattle
...,...,...,...
6073,POINT (-122.34716 47.61584),307.0,seattle
6074,POINT (-122.37198 47.70236),145.0,seattle
6075,POINT (-122.34887 47.61576),153.0,seattle
6076,POINT (-122.35028 47.62791),186.0,seattle


In [8]:
airbnb = pd.concat([airbnb_ny, airbnb_st])
airbnb

,geometry,price,mie_nazwa
0,POINT (-73.95512 40.68535),72.0,new_york
1,POINT (-73.99454 40.66265),253.0,new_york
2,POINT (-73.95342 40.70935),81.0,new_york
3,POINT (-73.94255 40.80107),65.0,new_york
4,POINT (-73.94759 40.78778),60.0,new_york
...,...,...,...
6073,POINT (-122.34716 47.61584),307.0,seattle
6074,POINT (-122.37198 47.70236),145.0,seattle
6075,POINT (-122.34887 47.61576),153.0,seattle
6076,POINT (-122.35028 47.62791),186.0,seattle


In [9]:



def add_airbnb_to_osmnx_nodes(
    airbnb: gpd.GeoDataFrame,
    nodes: gpd.GeoDataFrame,
    edges: gpd.GeoDataFrame,
    city_name: str,
):
    with open(OSMNX_ALL_ATTRIBUTES_LOCATION) as f:
        all_attributes = json.load(f)
    osmnx_graph = OSMnxGraph(
        airbnb.loc[airbnb["mie_nazwa"] == city_name, :],
        nodes,
        edges,
        all_attributes,
        y_column_name="price",
    )
    osmnx_graph._aggregate(element_type="node", aggregation_method="mean")
    nodes = osmnx_graph.gdf_nodes
    nodes["price"] = nodes["price"].apply(lambda x: None if x == 0.0 else x)
    return nodes


def create_gdfs(city_name: str):
    osmnx_nodes = gpd.read_parquet(
        os.path.join(ORGANIZED_GRAPHS_LOCATION, city_name, "nodes.parquet")
    )
    osmnx_nodes = osmnx_nodes.reset_index()
    osmnx_nodes.index.names = ["node_id"]
    osmnx_nodes["x"] = osmnx_nodes["geometry"].x
    osmnx_nodes["y"] = osmnx_nodes["geometry"].y

    osmnx_edges = gpd.read_parquet(
        os.path.join(ORGANIZED_GRAPHS_LOCATION, city_name, "edges.parquet")
    )
    osmnx_edges = osmnx_edges.reset_index().rename(columns={"index": "edge_id"})
    osmnx_edges.index.names = ["edge_id"]
    if "level_4" in osmnx_edges.columns:
        osmnx_edges = osmnx_edges.drop("level_4", axis=1)
    osmnx_edges = osmnx_edges.fillna(0)

    assert osmnx_nodes.crs == osmnx_edges.crs
    assert osmnx_nodes.crs == airbnb.crs

    osmnx_nodes = add_airbnb_to_osmnx_nodes(
        airbnb=airbnb,
        nodes=osmnx_nodes,
        city_name=city_name,
        edges=osmnx_edges,
    )

    hexes_years_folder = os.path.join(ORGANIZED_HEXES_LOCATION, city_name)

    subfolders = [
        int(f)
        for f in os.listdir(hexes_years_folder)
        if os.path.isdir(os.path.join(hexes_years_folder, f))
    ]
    highest_year = subfolders[np.argmax(subfolders)]

    hexes: gpd.GeoDataFrame = gpd.read_parquet(
        os.path.join(
            ORGANIZED_HEXES_LOCATION,
            f"{city_name}/{highest_year}/h9/count-embedder/dataset.parquet",
        )
    )

    hexes = hexes.rename(columns={"region_id": "h3_id"}).rename_axis(
        "region_id", axis=0
    )

    return dict(osmnx_nodes=osmnx_nodes, osmnx_edges=osmnx_edges, hexes=hexes)


print("Creating gdfs...")
gdfs_dict = {
    city_name: create_gdfs(city_name) for city_name in tqdm(["new_york", "seattle"])
}

Creating gdfs...


100%|██████████| 2/2 [00:01<00:00,  1.00it/s]


In [10]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for gdf_for_city in gdfs_dict.values():
        gdf_for_city["controller"] = GraphLayerController(
            gdf_for_city["hexes"],
            gdf_for_city["osmnx_nodes"],
            gdf_for_city["osmnx_edges"],
        )

In [11]:
def patch_hexes_with_y(
    osmnx_nodes: gpd.GeoDataFrame,
    hexes: gpd.GeoDataFrame,
    controller: GraphLayerController,
):
    virtual_edges = controller.get_virtual_edges_to_hexes(SourceType.OSMNX_NODES)
    hexes_with_y = cast(
        gpd.GeoDataFrame,
        hexes.merge(
            virtual_edges.merge(osmnx_nodes, left_on="source_id", right_index=True)[
                ["region_id", "price"]
            ]
            .groupby("region_id")
            .mean(),
            left_index=True,
            right_index=True,
            how="left",
        ),
    )
    grouped_not_na = hexes_with_y[~hexes_with_y["price"].isna()]
    df_grouped_not_na = gpd.GeoDataFrame(grouped_not_na)
    q1 = df_grouped_not_na["price"].quantile(0.25)
    q2 = df_grouped_not_na["price"].quantile(0.5)
    q3 = df_grouped_not_na["price"].quantile(0.75)
    bins = [0, q1, q2, q3, float("inf")]
    label_names = [1, 2, 3, 4]

    df_grouped_not_na["price_class"] = pd.cut(
        df_grouped_not_na["price"], bins=bins, labels=label_names, right=True
    )

    grouped_isna = hexes_with_y[hexes_with_y["price"].isna()]
    grouped_na = gpd.GeoDataFrame(grouped_isna)
    grouped_na["price_class"] = 0

    hexes_with_y = pd.concat([df_grouped_not_na, grouped_na])
    hexes_with_y.drop("price", axis=1, inplace=True)
    hexes_with_y = hexes_with_y.set_crs("EPSG:4326")
    display(hexes_with_y)
    controller.hexes_gdf = hexes_with_y
    controller._hexes_centroids_gdf = controller._create_hexes_centroids_gdf()

In [12]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for gdfs in gdfs_dict.values():
        patch_hexes_with_y(gdfs["osmnx_nodes"], gdfs["hexes"], gdfs["controller"])

,h3_id,geometry,amenity_gym,building_garages,landuse_gravel,sport_ultimate,office_bakery,natural_shrubbery,landuse_gress,building_guard_cabin,...,historic_heritage,building_government,aeroway_navigationaid,historic_park,historic_train_station,shop_hobby,building_floating_home,amenity_vacuum_cleaner,building_castle,price_class
region_id,,,,,,,,,,,,,,,,,,,,,
15,892a1000047ffff,"POLYGON ((-73.79056 40.85996, -73.79267 40.859...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
22,892a1000067ffff,"POLYGON ((-73.78814 40.85208, -73.79025 40.851...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
93,892a100024bffff,"POLYGON ((-73.82708 40.86168, -73.82920 40.860...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
97,892a100025bffff,"POLYGON ((-73.82504 40.86438, -73.82715 40.863...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
138,892a1000383ffff,"POLYGON ((-73.78796 40.84679, -73.79007 40.845...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10749,892a10776bbffff,"POLYGON ((-74.00607 40.66922, -74.00817 40.668...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10751,892a10776c7ffff,"POLYGON ((-74.01482 40.67426, -74.01692 40.673...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10752,892a10776cbffff,"POLYGON ((-74.02133 40.67672, -74.02343 40.675...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,h3_id,geometry,amenity_gym,building_garages,landuse_gravel,sport_ultimate,office_bakery,natural_shrubbery,landuse_gress,building_guard_cabin,...,historic_heritage,building_government,aeroway_navigationaid,historic_park,historic_train_station,shop_hobby,building_floating_home,amenity_vacuum_cleaner,building_castle,price_class
region_id,,,,,,,,,,,,,,,,,,,,,
35,8928d540103ffff,"POLYGON ((-122.27904 47.65803, -122.27784 47.6...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
37,8928d54010bffff,"POLYGON ((-122.27525 47.65980, -122.27406 47.6...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
38,8928d54010fffff,"POLYGON ((-122.27924 47.66089, -122.27805 47.6...",0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
40,8928d540117ffff,"POLYGON ((-122.28282 47.65627, -122.28163 47.6...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
41,8928d54011bffff,"POLYGON ((-122.27505 47.65695, -122.27385 47.6...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4198,8928d5cdbcbffff,"POLYGON ((-122.39215 47.49291, -122.39096 47.4...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4199,8928d5cdbcfffff,"POLYGON ((-122.39613 47.49399, -122.39494 47.4...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4200,8928d5cdbd3ffff,"POLYGON ((-122.39572 47.48827, -122.39453 47.4...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
def create_graph_data(
    osmnx_nodes,
    osmnx_edges,
    hexes,
    controller: GraphLayerController,
    use_hexes_attr: bool,
    use_ortophoto: bool,
    use_all_columns = True,
    columns_to_take = []
    
):

    edges_attr_columns = osmnx_edges.columns[
        ~osmnx_edges.columns.isin(["u", "v", "key", "geometry"])
    ]
    nodes_attr_columns = osmnx_nodes.columns[
        ~osmnx_nodes.columns.isin(["geometry", "x", "y", "osmid"])
    ]
    if use_all_columns:
        hexes_attr_columns = (
            hexes.columns[~hexes.columns.isin(["geometry", "h3_id", "price_class"])]
            if use_hexes_attr
            else []
        )
    else:
        hexes_attr_columns = columns_to_take

    data = create_hetero_data(
        controller,
        hexes_attrs_columns_names=hexes_attr_columns,
        osmnx_edge_attrs_columns_names=edges_attr_columns,
        osmnx_node_attrs_columns_names=nodes_attr_columns,
        virtual_edge_attrs_columns_names=[],
        hexes_y_columns_names=["price_class"],
    )
    return data


graph_data_dict = {
    city_name: create_graph_data(**gdfs, use_ortophoto=False, use_hexes_attr=True)
    for city_name, gdfs in gdfs_dict.items()
}

In [14]:
def create_tabular_data(
    hexes: pd.DataFrame,
    controller: GraphLayerController,
    use_hexes_attr: bool,
    use_ortophoto: bool,
    use_all_columns = True,
    columns_to_take = []
):
    assert use_ortophoto or use_hexes_attr, "Provide at least one data source"
    if use_all_columns:
        hexes_attr_columns = (
            hexes.columns[~hexes.columns.isin(["geometry", "h3_id"])]
            if use_hexes_attr
            else []
        )
    else:
        hexes_attr_columns = columns_to_take
        
    hexes_y_columns_names = ["price_class"]

    X = hexes[hexes_attr_columns]
    y = controller.hexes_centroids_gdf[hexes_y_columns_names]

    return {"X": X, "y": y}


tabular_data_dict = {
    city_name: create_tabular_data(
        gdfs["hexes"], gdfs["controller"], use_ortophoto=False, use_hexes_attr=True
    )
    for city_name, gdfs in gdfs_dict.items()
}

In [15]:
def shift_elements_right(lst):
    shifted_lst = [lst[-1]] + lst[:-1]
    return shifted_lst


cities_names_list = list(graph_data_dict.keys())
cities_names_list.sort(key=lambda x: str(x))

# val + test
folds_tuples = list(zip(shift_elements_right(cities_names_list), cities_names_list))
display(folds_tuples)

[('seattle', 'new_york'), ('new_york', 'seattle')]

In [16]:
display(tabular_data_dict["seattle"]["y"].iloc[:, 0].unique())
display(tabular_data_dict["seattle"]["y"].iloc[:, 0].unique())

array([3, 1, 4, 2, 0])

array([3, 1, 4, 2, 0])

In [17]:
from typing import cast
from wandb.sdk.wandb_run import Run
import torch


def run_k_fold_graph_data(closure_config, sweep_id, hex_attr_config=None):
    # pass external config (i.e. what attributes are used in the data), closure to avoid passing it to the function directly
    def wrapped():
        run: Run = cast(Run, wandb.init())

        config = wandb.config

        for k, v in closure_config.items():
            run.log({k: 1 if v else 0})

        run.log({"data_structure": "graph"})
        if hex_attr_config is not None:
            run.log({"hex_features": f"top_{hex_attr_config['NUM_FEATURES']}_percent_{hex_attr_config['IN_PERCENT']}"})
        else:
            run.log({"hex_features": "all"})

        # create hparams
        if hasattr(config, "lin_layer_size") and hasattr(config, "num_lin_layers"):
            lin_layer_sizes = [config.lin_layer_size] * config.num_lin_layers
        else:
            lin_layer_sizes = config.lin_layer_sizes
        hparams = {
            "hidden_channels": config.hidden_channels,
            "lr": config.learning_rate,
            "num_conv_layers": config.num_conv_layers,
            "lin_layer_sizes": lin_layer_sizes,
            "weight_decay": config.weight_decay,
        }

        epochs = config.epochs

        aucs = []
        accuracies = []
        f1s = []

        fold_group_id = generate_id()

        # log data as artifact if no data was logged in the sweep before
        # dataset is uploaded only on the first run in sweep, because it does not change across runs in sweep
        # in wandb, dataset will be visible on the first run in the sweep
        artifact_path = os.path.join(TRAIN_SAVE_DIR, f"graph_data_{sweep_id}.pkl")
        if not os.path.exists(artifact_path):
            dump(
                graph_data_dict,
                artifact_path,
                protocol=5,
            )
            artifact = wandb.Artifact(
                name="graph_data", type="dataset", metadata=closure_config
            )
            artifact.add_file(local_path=artifact_path)
            run.log_artifact(artifact)

        # run k-fold
        for index, (_, test_city_name) in enumerate(folds_tuples):
            # prepare data
            train_data = [
                v.to("cpu").clone()
                for k, v in graph_data_dict.items()
                if k != test_city_name
            ]
            test_data = graph_data_dict[test_city_name].to("cpu").clone()

            # run training with checkpointing on lowest val_loss, return test metrics for the best model and its path
            # builtin preprocessing - scaling to N(0, 1)
            auc, accuracy, f1, model_path = train(
                train_data=train_data,
                val_data=None,
                test_data=test_data,
                epochs=epochs,
                hparams=hparams,
                train_save_dir=TRAIN_SAVE_DIR,
                num_classes=torch.unique(train_data[0]["hex"].y).shape[0],
            )

            # logging - single fold
            run.log_model(
                path=model_path,
                name=f"model_{fold_group_id}_fold_{index}",
            )
            run.log({f"auc_fold_{index}": auc})
            run.log({f"accuracy_fold_{index}": accuracy})
            run.log({f"f1_fold_{index}": f1})

            aucs.append(auc)
            accuracies.append(accuracy)
            f1s.append(f1)

        # logging - summary statistics
        mean_auc = sum(aucs) / len(aucs)
        mean_accuracy = sum(accuracies) / len(accuracies)
        mean_f1 = sum(f1s) / len(f1s)
        run.log({"mean_auc": mean_auc})
        run.log({"mean_accuracy": mean_accuracy})
        run.log({"mean_f1": mean_f1})

    return wrapped


def run_k_fold_tabular_data(closure_config, sweep_id, hex_attr_config=None):
    # analogously to the graph data, but for tabular data
    def wrapped():
        run: Run = cast(Run, wandb.init())

        config = wandb.config

        for k, v in closure_config.items():
            run.log({k: 1 if v else 0})

        run.log({"data_structure": "tabular"})
        if hex_attr_config is not None:
            run.log({"hex_features": f"top_{hex_attr_config['NUM_FEATURES']}_percent_{hex_attr_config['IN_PERCENT']}"})
        else:
            run.log({"hex_features": "all"})


        hparams = {}
        hparams["C"] = config["C"]
        solver, penalty = config["solver_penalty"].split(";")
        hparams["solver"] = solver
        if penalty == "None":
            penalty = None
        hparams["penalty"] = penalty

        aucs = []
        accuracies = []
        f1s = []

        fold_group_id = generate_id()

        # log data as artifact
        artifact_path = os.path.join(TRAIN_SAVE_DIR, f"tabular_data_{sweep_id}.pkl")

        if not os.path.exists(artifact_path):
            dump(
                tabular_data_dict,
                artifact_path,
                protocol=5,
            )
            artifact = wandb.Artifact(
                name="tabular_data", type="dataset", metadata=closure_config
            )
            artifact.add_file(local_path=artifact_path)
            run.log_artifact(artifact)

        timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

        for index, test_city_name in enumerate(cities_names_list):
            # the original setup used standard scaling, but it zeroes out all coefficients in the logistic regression
            # see https://wandb.ai/gradient_pwr/airbnb-downstream-task/sweeps/ow0algrs/overview
            # scaler = StandardScaler()
            X = pd.concat(
                [
                    m["X"]
                    for key, m in tabular_data_dict.items()
                    if key != test_city_name
                ]
            ).to_numpy()
            y = (
                pd.concat(
                    [
                        m["y"]
                        for key, m in tabular_data_dict.items()
                        if key != test_city_name
                    ]
                )
                .to_numpy()
                .ravel()
            )

            # X = scaler.fit_transform(X)

            logistic_regression = LogisticRegression(
                C=hparams["C"],
                # solver=hparams["solver"],
                # penalty=hparams["penalty"],
                solver="newton-cholesky",
                penalty="l2",
                dual=False,
                tol=1e-4,
                fit_intercept=True,
                intercept_scaling=1,
                class_weight="balanced",
                random_state=1124,
                max_iter=2,
                multi_class="auto",
                warm_start=False,
                n_jobs=-1,
                l1_ratio=0.5,
            )
            logistic_regression.fit(X, y)

            test_X = tabular_data_dict[test_city_name]["X"].to_numpy()
            # test_X = scaler.transform(test_X)
            test_y = tabular_data_dict[test_city_name]["y"].to_numpy().ravel()

            encoder = OneHotEncoder(sparse=False)
            test_y_ohe = encoder.fit_transform(test_y.reshape(-1, 1))

            y_pred = logistic_regression.predict(test_X)
            y_proba = logistic_regression.predict_proba(test_X)

            auc = roc_auc_score(
                test_y_ohe, y_proba, average="weighted", multi_class="ovr"
            )
            accuracy = (y_pred == test_y).mean()
            f1 = f1_score(
                test_y,
                y_pred,
                average="weighted",
            )

            model_dir = os.path.join(TRAIN_SAVE_DIR, timestamp)

            os.makedirs(model_dir, exist_ok=True)

            model_path = os.path.join(
                model_dir, f"model_{fold_group_id}_fold_{index}.pkl"
            )

            with open(model_path, "wb") as f:
                dump(logistic_regression, f, protocol=5)

            run.log_model(
                path=model_path,
                name=f"model_{fold_group_id}_fold_{index}",
            )
            run.log({f"auc_fold_{index}": auc})
            run.log({f"accuracy_fold_{index}": accuracy})
            run.log({f"f1_fold_{index}": f1})

            aucs.append(auc)
            accuracies.append(accuracy)
            f1s.append(f1)

        mean_auc = sum(aucs) / len(aucs)
        mean_accuracy = sum(accuracies) / len(accuracies)
        mean_f1 = sum(f1s) / len(f1s)
        run.log({"mean_auc": mean_auc})
        run.log({"mean_accuracy": mean_accuracy})
        run.log({"mean_f1": mean_f1})

    return wrapped


def run_sweep_graph_data(config, hex_attr_config=None):
    try:
        wandb.login(key=WANDB_API_KEY)
        sweep_id = wandb.sweep(
            WANDB_SWEEP_PARAMS_GRAPH_DATA, project="airbnb-downstream-task"
        )

        wandb.agent(
            sweep_id,
            function=run_k_fold_graph_data(config, sweep_id, hex_attr_config),
            count=SWEEP_RUNS_COUNT,
        )
    except Exception as e:
        print(e)
        wandb.finish()
        wandb.sweep
        raise e


def run_sweep_tabular_data(config, hex_attr_config=None):
    try:
        wandb.login(key=WANDB_API_KEY)

        sweep_id = wandb.sweep(
            WANDB_SWEEP_PARAMS_TABULAR_DATA, project="airbnb-downstream-task"
        )

        wandb.agent(
            sweep_id,
            function=run_k_fold_tabular_data(config, sweep_id, hex_attr_config),
            count=SWEEP_RUNS_COUNT,
        )
    except Exception as e:
        print(e)
        wandb.finish()
        raise e

In [18]:
def derive_data_structure(attr_config):
    if attr_config["USE_OSMNX_ATTRS"]:
        return "graph"
    return "tabular"

configs_size = len(ATTRIBUTES_CONFIGURATIONS)
hex_configs_fi_size = len(HEX_ATTRS_FI)

if hex_configs_fi_size == 0:
    for index, attr_config in enumerate(ATTRIBUTES_CONFIGURATIONS):
        print("Sweep for config {}/{} in progress...".format(index + 1, configs_size))

        assert "USE_ORTOPHOTO" in attr_config, "Provide USE_ORTOPHOTO key"
        assert "USE_HEXES_ATTRS" in attr_config, "Provide USE_HEXES_ATTRS key"
        assert "USE_OSMNX_ATTRS" in attr_config, "Provide USE_OSMNX_ATTRS key"

        data_structure = derive_data_structure(attr_config)

        if data_structure == "graph":
            graph_data_dict = {
                city_name: create_graph_data(
                    **gdfs,
                    use_ortophoto=attr_config["USE_ORTOPHOTO"],
                    use_hexes_attr=attr_config["USE_HEXES_ATTRS"],
                )
                for city_name, gdfs in gdfs_dict.items()
            }
            run_sweep_graph_data(attr_config)
        elif data_structure == "tabular":
            tabular_data_dict = {
                city_name: create_tabular_data(
                    gdfs["hexes"],
                    gdfs["controller"],
                    use_ortophoto=attr_config["USE_ORTOPHOTO"],
                    use_hexes_attr=attr_config["USE_HEXES_ATTRS"],
                )
                for city_name, gdfs in gdfs_dict.items()
            }
            run_sweep_tabular_data(attr_config)
        else:
            raise ValueError("Unknown data structure")
else:
    for index, hex_fi_config in enumerate(HEX_ATTRS_FI):
        assert "NUM_FEATURES" in hex_fi_config, "Provide NUM_FEATURES key"
        assert "IN_PERCENT" in hex_fi_config, "Provide IN_PERCENT key"
        try:
            hex_features = pd.read_json(f"{HEX_FI_LOCATION}/airbnb_top_{hex_fi_config['NUM_FEATURES']}_percent_{hex_fi_config['IN_PERCENT']}.json")
            hex_features = hex_features['top_values'].tolist()
            tabular_data_dict = {
                    city_name: create_tabular_data(
                        gdfs["hexes"], gdfs["controller"], use_ortophoto=False, use_hexes_attr=True, use_all_columns=False, columns_to_take=hex_features
                    )
                    for city_name, gdfs in gdfs_dict.items()
                }
            
            graph_data_dict = {
                city_name: create_graph_data(**gdfs, use_ortophoto=False, use_hexes_attr=True, use_all_columns=False, columns_to_take=hex_features)
                for city_name, gdfs in gdfs_dict.items()
            }
        except FileNotFoundError:
            print(f"File not found for feature importance values for hexes in path {HEX_FI_LOCATION}/airbnb_top_{hex_fi_config['NUM_FEATURES']}_percent_{hex_fi_config['IN_PERCENT']}.json")
            print("Taking all hex columns...")
            tabular_data_dict = {
                    city_name: create_tabular_data(
                        gdfs["hexes"], gdfs["controller"], use_ortophoto=False, use_hexes_attr=True, use_all_columns=True)
                    for city_name, gdfs in gdfs_dict.items()
                }
            
            graph_data_dict = {
                city_name: create_graph_data(**gdfs, use_ortophoto=False, use_hexes_attr=True, use_all_columns=True)
                for city_name, gdfs in gdfs_dict.items()
            }
        
        for index, attr_config in enumerate(ATTRIBUTES_CONFIGURATIONS):
            print("Sweep for config {}/{} in progress...".format(index + 1, configs_size))

            assert "USE_ORTOPHOTO" in attr_config, "Provide USE_ORTOPHOTO key"
            assert "USE_HEXES_ATTRS" in attr_config, "Provide USE_HEXES_ATTRS key"
            assert "USE_OSMNX_ATTRS" in attr_config, "Provide USE_OSMNX_ATTRS key"

            data_structure = derive_data_structure(attr_config)

            if data_structure == "graph":
                graph_data_dict = {
                    city_name: create_graph_data(
                        **gdfs,
                        use_ortophoto=attr_config["USE_ORTOPHOTO"],
                        use_hexes_attr=attr_config["USE_HEXES_ATTRS"],
                    )
                    for city_name, gdfs in gdfs_dict.items()
                }
                run_sweep_graph_data(attr_config, hex_fi_config)
            elif data_structure == "tabular":
                tabular_data_dict = {
                    city_name: create_tabular_data(
                        gdfs["hexes"],
                        gdfs["controller"],
                        use_ortophoto=attr_config["USE_ORTOPHOTO"],
                        use_hexes_attr=attr_config["USE_HEXES_ATTRS"],
                    )
                    for city_name, gdfs in gdfs_dict.items()
                }
                run_sweep_tabular_data(attr_config, hex_fi_config)
            else:
                raise ValueError("Unknown data structure")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Sweep for config 1/1 in progress...


wandb: Currently logged in as: grymar (gradient_pwr). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/grymar/.netrc


Create sweep with ID: om79vbdm
Sweep URL: https://wandb.ai/gradient_pwr/airbnb-downstream-task/sweeps/om79vbdm


wandb: Agent Starting Run: viaq4o9s with config:
wandb: 	C: 0.000125751748869542
wandb: 	solver_penalty: sag;l2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/grymar/.netrc


Sweep for config 1/1 in progress...
Create sweep with ID: 8ljbcxgi
Sweep URL: https://wandb.ai/gradient_pwr/airbnb-downstream-task/sweeps/8ljbcxgi


wandb: Agent Starting Run: 332ot0m3 with config:
wandb: 	C: 0.012330358196413147
wandb: 	solver_penalty: liblinear;l2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/linear_model/_glm/_newton_solver.py:423: ConvergenceWarning: Newton solver did not converge after 2 iterations.
  warnings.warn(
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/linear_model/_glm/_newton_solver.py:423: ConvergenceWarning: Newton solver did not converge after 2 iterations.
  warnings.warn(
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/linear_model/_glm/_newton_solver.py:423: ConvergenceWarning: Newton solver did not converge after 2 iterations.
  warnings.warn(
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/linear_model/_glm/_newton_solver.py:423: ConvergenceWarning: Newton solver did not converge after 2 iterations.
  warnings

USE_HEXES_ATTRS,▁
USE_ORTOPHOTO,▁
USE_OSMNX_ATTRS,▁
accuracy_fold_0,▁
accuracy_fold_1,▁
auc_fold_0,▁
auc_fold_1,▁
f1_fold_0,▁
f1_fold_1,▁
mean_accuracy,▁
mean_auc,▁
